In [0]:
import torchvision
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
from PIL import Image
import os
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -r '/usr/local/lib/python3.6/dist-packages/tensorflow_datasets'

In [0]:
print(cuda.is_available())
print(cuda.device_count())

True
1


In [0]:
def getImages(directory, width, height):
  index = 0
  location = []
  for d in os.listdir(directory):
    image = (Image.open((directory + '/' + d)))
    location.append(image.resize((width,height)))
    index = index + 1
  return location


In [0]:
#Get all the images, and standardize size of the images using pillow

train_image_n=[]
train_image_p=[]
test_image_n=[]
test_image_p=[]
val_image_n=[]
val_image_p=[]

image_train_dir_n = '/content/drive/Shared drives/Data/chest_xray/Train/NORMAL'
image_train_dir_p = '/content/drive/Shared drives/Data/chest_xray/Train/PNEUMONIA'
image_test_dir_n = '/content/drive/Shared drives/Data/chest_xray/Test/NORMAL'
image_test_dir_p = '/content/drive/Shared drives/Data/chest_xray/Test/PNEUMONIA'
image_val_dir_n = '/content/drive/Shared drives/Data/chest_xray/val/NORMAL'
image_val_dir_p = '/content/drive/Shared drives/Data/chest_xray/val/PNEUMONIA'

train_image_n = getImages(image_train_dir_n, 32, 32)
train_image_p = getImages(image_train_dir_p, 32, 32)
test_image_n = getImages(image_test_dir_n, 32, 32)
test_image_p = getImages(image_test_dir_p, 32, 32)


In [0]:
files = os.listdir('/content/drive/Shared drives/Data/chest_xray/train/NORMAL')
files

In [0]:
def saveImages(imageArray, directory):
  index=0
  for image in imageArray:
    image.save(directory + '/image'+ str(index)+ '.jpg')
    index = index + 1

In [0]:
#Save images after preprocessing

saveImages(train_image_n, '/content/drive/Shared drives/Data/chest_xray_subset_2/Train/N')
saveImages(train_image_p, '/content/drive/Shared drives/Data/chest_xray_subset_2/Train/P')
saveImages(test_image_n, '/content/drive/Shared drives/Data/chest_xray_subset_2/Test/N')
saveImages(test_image_p, '/content/drive/Shared drives/Data/chest_xray_subset_2/Test/P')
# saveImages(val_image_n, '/content/drive/Shared drives/Data/chest_xray_subset/Val/N')
# saveImages(val_image_p, '/content/drive/Shared drives/Data/chest_xray_subset/Val/P')

NameError: ignored

In [0]:
#Load images from drive

image_train_dir = '/content/drive/Shared drives/Data/chest_x_ray_combined'
image_test_dir = '/content/drive/Shared drives/Data/chest_xray_subset_3/Test'
image_val_dir = '/content/drive/Shared drives/Data/chest_xray_subset_3/Val'


In [0]:
!rm -r '/content/drive/Shared drives/Data/chest_xray_subset_2'

In [0]:
!mkdir '/content/drive/Shared drives/Data/chest_xray_subset_2'

In [0]:
#Standardize size
resize = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])   #transforms.Resize((224,224))

#Format dataset to load into model
training_dataset = datasets.ImageFolder(image_train_dir, transform=resize)
validation_dataset = datasets.ImageFolder(image_val_dir, transform=resize)
testing_dataset = datasets.ImageFolder(image_test_dir, transform=resize)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
validate_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=32)


In [0]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
model = models.alexnet(pretrained=False)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [0]:
from collections import OrderedDict
for parameter in model.parameters():
    parameter.requires_grad = False

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                        ('relu', nn.ReLU()),
                                        ('drop', nn.Dropout(p=0.5)),
                                        ('fc2', nn.Linear(5000, 2)),
                                        ('output', nn.LogSoftmax(dim=1))]))

model.classifier = classifier

In [0]:
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):

        images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [0]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [0]:

def train_classifier():

        epochs = 15
        steps = 0
        print_every = 40

        model.to('cuda')

        for e in range(epochs):
        
            model.train()
    
            running_loss = 0
    
            for images, labels in iter(train_loader):
        
                steps += 1
        
                images, labels = images.to('cuda'), labels.to('cuda')
        
                optimizer.zero_grad()
        
                output = model.forward(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
        
                if steps % print_every == 0:
                
                    model.eval()
                
                    # Turn off gradients for validation, saves memory and computations
                    with torch.no_grad():
                        validation_loss, accuracy = validation(model, validate_loader, criterion)
            
                    print("Epoch: {}/{}.. ".format(e+1, epochs),
                          "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                          "Validation Loss: {:.3f}.. ".format(validation_loss/len(validate_loader)),
                          "Validation Accuracy: {:.3f}".format(accuracy/len(validate_loader)))
            
                    running_loss = 0
                    model.train()
                    
train_classifier() 

Epoch: 1/15..  Training Loss: 1.084..  Validation Loss: 0.007..  Validation Accuracy: 1.000
Epoch: 2/15..  Training Loss: 0.047..  Validation Loss: 0.019..  Validation Accuracy: 1.000
Epoch: 3/15..  Training Loss: 0.019..  Validation Loss: 0.069..  Validation Accuracy: 1.000
Epoch: 3/15..  Training Loss: 0.034..  Validation Loss: 0.037..  Validation Accuracy: 1.000
Epoch: 4/15..  Training Loss: 0.025..  Validation Loss: 0.019..  Validation Accuracy: 1.000
Epoch: 5/15..  Training Loss: 0.024..  Validation Loss: 0.000..  Validation Accuracy: 1.000
Epoch: 6/15..  Training Loss: 0.003..  Validation Loss: 0.023..  Validation Accuracy: 1.000
Epoch: 6/15..  Training Loss: 0.011..  Validation Loss: 0.001..  Validation Accuracy: 1.000
Epoch: 7/15..  Training Loss: 0.008..  Validation Loss: 0.000..  Validation Accuracy: 1.000
Epoch: 8/15..  Training Loss: 0.003..  Validation Loss: 0.002..  Validation Accuracy: 1.000
Epoch: 9/15..  Training Loss: 0.000..  Validation Loss: 0.006..  Validation Accu

In [0]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to('cuda')

    with torch.no_grad():
    
        accuracy = 0
        model_positive = 0
        true_positive = 0
        false_negative = 0
        for images, labels in iter(test_loader):
    
            images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images)

            probabilities = torch.exp(output)
            vals = torch.sum(probabilities.max(dim=1)[1]).type(torch.FloatTensor)
            model_positive +=vals.item()
            #actual_positive += torch.sum(labels.data).item()

            for pred, label in zip (probabilities.max(dim=1)[1], labels):
              if (pred.item() == 1 and label.item() == 1):
                true_positive += 1
              if (pred.item() == 0 and label.item() == 1):
                false_negative += 1


            equality = (labels.data == probabilities.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()

        print(true_positive / model_positive) #Precision
        print (true_positive / (true_positive + false_negative))  #Recall
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    #Test Accuracy
        
        
test_accuracy(model, test_loader)

0.7832310838445807
0.982051282051282
Test Accuracy: 0.823437511920929


In [0]:
def save_checkpoint(model):

    model.class_to_idx = training_dataset.class_to_idx

    checkpoint = {'arch': "vgg16",
                  'class_to_idx': model.class_to_idx,
                  'model_state_dict': model.state_dict()
                 }

    torch.save(checkpoint, '/content/drive/Shared drives/Data/colab_code/alexnet.pth')
    
save_checkpoint(model)

In [0]:
from collections import OrderedDict

# Function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    
    if checkpoint['arch'] == 'vgg16':
        
        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                            ('relu', nn.ReLU()),
                                            ('drop', nn.Dropout(p=0.5)),
                                            ('fc2', nn.Linear(5000, 2)),
                                            ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model

model = load_checkpoint('/content/drive/Shared drives/Data/colab_code/vgg.pth')

In [0]:
test_accuracy(model, test_loader)

Test Accuracy: 0.893750011920929
